# Caso Financiero - Banco de Colombia

## Business Understanding 




## Data understanding

### Data description report

En la exploración de los datos nos encontramos con una base de datos que contiene registros de 47871 clientes de tarjetas de un banco que maneja tres franquicias: visa, mastercard y una independiente del banco. Las variables que nos encontramos en cada uno de estos registros nos indican pertenencia a un grupo (del cual no tenemos información), estadística sobre las compras del usuario, la franquicia que usa, uso nacional o internacional, hora del día, día de la semana y tipo de comercio donde más compras realizó, totalizando 25 variables. 

El formato predominante de los datos es el formato de porcentaje, pues es así como está determinado el uso de franquicias sobre el uso total de todas, el uso nacional o internacional con las franquicias y en el total, el uso en cada rango horario y el uso en cada día de la semana; por otro lado, los datos como el mínimo, máximo y promedio de compra se encuentra en términos nominales de dinero, presumiblemente pesos colombianos y el tipo de comercio preferido como una variable categórica.

### Data exploration

Para la exploración de datos decidimos separar los registros en función del sitio preferido de los clientes, pues graficar en función de cada registro resulta inviable por la cantidad que tenemos y además carente de significado para el análisis, en cambio, los sitios preferidos si nos dan una visión significativa para las demás variables que tenemos. Por lo tanto será así como exploremos los datos y los trabajemos más adelante, como forma de exploración hicimos previamente un gráfico de barras donde se representa el valor total de compras en cada comercio usando visas en territorio nacional:

<img src='Imagen4.png'>

Aquí vemos entonces que con visas en territorio nacional los clientes del banco usan mayormente sus tarjetas para comprar en SUPERMERCADOS Y TIENDAS EXPRESS, ALMACEN POR DEPARTAMENTO CON SUPERMERCADO, ALMACENES DE VESTUARIO Y ACCESORIO PARA LA FAMILIA, ALAMCENES DE CALZADO, etc.

Este modo de ver la información nos resulta particularmente útil para poder sugerir al banco que tipo de promociones lanzar, pues podremos determinar que combinación de variables resultará más efectiva por la cantidad de clientes interesados en aprovecharlas gracias a esta observación y análisis de su comportamiento.

### Data quality

No encontramos errores para corregir en la base de datos en sí, sin embargo hay varibles que no serán tomadas en cuenta para el desarrollo del análisis de los datos por su carencia de significado para el mismo, tal como el grupo del cliente, el mínimo, máximo y desviación estándar de las compras, porcentaje nacional e internacional total, debido a que ya tenemos la misma información con las franquicias y de los sitios de consumo excluiremos "SIN NOMBRE".

### Data preparation

Primero, importamos la lobrerías que nos serán útiles para el desarrollo del proyecto:

In [1]:
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

Importamos los datos contenidos en el documento de excel infobanca.xlsx

In [2]:
infobanca= pd.read_excel('infobanca.xlsx')
#infobanca= pd.read_excel("C:\\Users\\migue\\Documents\\GitHub\\Fintech_and_Machine_Learning\\Caso1\\infobanca.xlsx")
infobanca_original=infobanca

#### Data construction

Ahora creamos columna de total de ventas multiplicando el promedio por transacción por el número de transacciones con el fin de tener un número total aproximado del consumo de cada uno de los usuarios:

In [3]:
infobanca['Total_ventas']=infobanca['promedio_por_transaccion']*infobanca['Numero_de_transacciones']

#### Data cleaning

Eliminamos las columnas mencionadas previamente en el reporte de calidad de los datos y además eliminamos la columna del promedio por transacción, pues al tener el total consumido por cada usuario, no lo necesitamos de ahora en adelante. Eliminamos también los registros de usuarios cuyo sitio preferido haya sido "SIN NOMBRE":

In [4]:
coldt=['CLIENTE','grupo_de_cliente','Numero_de_transacciones','promedio_por_transaccion','transaccion_minima','transaccion_maxima','desviacion_estandar_por_transaccion','porcentaje_nacional_total','porcentaje_internacional_total']
infobanca= infobanca.drop(coldt, axis=1)
infobanca= infobanca[infobanca['Sitio_consumo_masfrecuente'] !='SIN NOMBRE']
infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Sitio_consumo_masfrecuente,Total_ventas
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,CLINICAS - HOSPITALES,459930.3
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.0,0.0,0.333333,0.0,0.0,MERCADEO DIRECTO - COMERCIANTES DE VENTAS TELE...,1748000.1
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,"DROGUERIAS, FARMACIAS, TIENDAS NATURISTAS",1849641.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.4,0.6,0.0,0.0,0.600000,0.2,0.0,0.100000,0.0,0.1,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,1444671.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,321764.0


#### Data formatting

Multiplicamos total venta por los porcentajes del DF con el fin de deshacernos de los porcentajes y en vez de eso tener valores netos en pesos colombianos de las ventas hechas con cada franquicia, hora del día, etc.

In [5]:
colum = list(infobanca.columns)
colum.remove('Sitio_consumo_masfrecuente')
colum.remove('Total_ventas')
for i in colum:
  infobanca[i] = infobanca[i]*infobanca['Total_ventas']
infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Sitio_consumo_masfrecuente,Total_ventas
0,0.0,459930.3,0.0,0.0,0.0,0.0,0.0,0.0,459930.3,0.0,0.0,459930.3,0.0,0.0,0.0,0.0,CLINICAS - HOSPITALES,459930.3
1,1748000.1,0.0,0.0,0.0,0.0,0.0,0.0,1748000.1,0.0,0.0,1165333.4,0.0,0.0,582666.7,0.0,0.0,MERCADEO DIRECTO - COMERCIANTES DE VENTAS TELE...,1748000.1
2,1849641.0,0.0,0.0,0.0,0.0,0.0,1849641.0,0.0,0.0,0.0,0.0,0.0,1849641.0,0.0,0.0,0.0,"DROGUERIAS, FARMACIAS, TIENDAS NATURISTAS",1849641.0
3,0.0,0.0,1444671.0,0.0,0.0,0.0,577868.4,866802.6,0.0,0.0,866802.6,288934.2,0.0,144467.1,0.0,144467.1,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,1444671.0
4,321764.0,0.0,0.0,0.0,0.0,0.0,0.0,321764.0,0.0,321764.0,0.0,0.0,0.0,0.0,0.0,0.0,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,321764.0


Ahora cambiamos la forma de la base de datos, pues en vez de tener un cliente en cada fila, tendremos cada uno de los sitios y las variables que quedarán registradas serán las sumatorias del total de dinero invertido en cada sitio bajo cada criterio, tal como las compras con visa nacional o internacional, la hora del día de la transacción, etc.

Esta será la base de datos con las que iniciemos la preparación de los datos para la aplicación del metodo de clustering K-means.

In [6]:
columnas=list(infobanca.columns.values)
columnas.remove('Total_ventas')
suma_por_sitio=infobanca.groupby('Sitio_consumo_masfrecuente')[columnas].sum()
infobanca=suma_por_sitio

infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",1.030147e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,1.030147e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.801470e+05,0.000000e+00,0.000000e+00,0.000000e+00,4.500000e+05
AEROLINEAS,2.186256e+09,7.111309e+08,1.435605e+09,4.533029e+08,31649905.85,97490739.12,1.491057e+09,2.369273e+09,1.055105e+09,4.472980e+08,6.592852e+08,7.800953e+08,8.038844e+08,7.799944e+08,8.536865e+08,5.911916e+08
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,4.731503e+07,5.798083e+06,2.098882e+07,2.363915e+06,161590.00,442522.84,2.166422e+07,3.781246e+07,1.759329e+07,1.168837e+07,5.219402e+06,1.181228e+07,1.345278e+07,1.116347e+07,7.654751e+06,1.607891e+07
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,6.206000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,6.206000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.206000e+05,0.000000e+00
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,1.400711e+09,1.114064e+08,5.258823e+08,5.848946e+07,8172345.31,32829322.66,6.804893e+08,1.265260e+09,1.917410e+08,1.247545e+08,3.298423e+08,3.639641e+08,3.316032e+08,3.434155e+08,3.637598e+08,2.801513e+08


In [7]:
colum= infobanca.columns
for i in colum:
    infobanca[i]=infobanca[i]/1000000

In [8]:
infobanca=round(infobanca,2)

In [9]:
infobanca

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",1.03,0.00,0.00,0.00,0.00,0.00,1.03,0.00,0.00,0.00,0.00,0.58,0.00,0.00,0.00,0.45
AEROLINEAS,2186.26,711.13,1435.61,453.30,31.65,97.49,1491.06,2369.27,1055.11,447.30,659.29,780.10,803.88,779.99,853.69,591.19
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,47.32,5.80,20.99,2.36,0.16,0.44,21.66,37.81,17.59,11.69,5.22,11.81,13.45,11.16,7.65,16.08
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,0.62,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.62,0.00
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,1400.71,111.41,525.88,58.49,8.17,32.83,680.49,1265.26,191.74,124.75,329.84,363.96,331.60,343.42,363.76,280.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"VENTA DE ARTICULOS MEDICOS, ODONTOLOGICOS, ORTOPEDICOS",36.03,0.00,36.46,3.31,0.00,0.00,28.37,40.99,6.44,3.08,5.86,11.79,15.43,7.36,10.67,21.61
VENTA DE COMIDA RAPIDA,48.78,8.73,92.56,6.54,0.97,0.85,28.43,95.84,34.18,31.80,22.45,18.92,18.23,21.00,19.37,26.67
VENTA DE BOTES Y ACCESORIOS PARA ACTIVIDADES ACUATICAS,0.89,0.00,4.35,0.00,0.00,0.00,2.40,2.83,0.00,0.59,1.40,0.00,0.45,0.26,0.00,2.53


#### Escalar datos

Escalamos la base de datos con standard sca `StandardScaler` para normalizar los datos y creamos el índice `Sitio_consumo_masfrecuente`

In [10]:
infobanca_escalado=StandardScaler().fit_transform(infobanca)

# DataFrame con los datos escalados
df_infobanca_scaled = pd.DataFrame(
    infobanca_escalado,
    columns=infobanca.columns
)

# Copiamos los nombres de los sitios
df_infobanca_scaled["Sitio_consumo_masfrecuente"] = infobanca.index

# Establecer la columna Sitio_consumo_masfrecuente como índice
df_infobanca_scaled= df_infobanca_scaled.set_index("Sitio_consumo_masfrecuente")

df_infobanca_scaled.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",-0.405491,-0.288622,-0.356500,-0.295740,-0.334358,-0.159009,-0.413602,-0.397871,-0.362379,-0.318788,-0.406218,-0.426617,-0.425587,-0.428480,-0.424400,-0.364972
AEROLINEAS,4.001300,3.815425,1.414048,3.221043,0.978403,1.108289,3.111974,2.487574,3.576975,1.826478,2.898515,3.336319,3.377035,3.281451,3.558383,1.987535
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,-0.312142,-0.255150,-0.330612,-0.277430,-0.327722,-0.153289,-0.364789,-0.351824,-0.296705,-0.262722,-0.380052,-0.372407,-0.361964,-0.375398,-0.388710,-0.302729
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,-0.406318,-0.288622,-0.356500,-0.295740,-0.334358,-0.159009,-0.414572,-0.397871,-0.362379,-0.318788,-0.406218,-0.429417,-0.425587,-0.428480,-0.421507,-0.366764
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,2.417140,0.354343,0.292072,0.158036,0.004513,0.267757,1.194076,1.143041,0.353501,0.279517,1.247126,1.327508,1.142993,1.204957,1.272676,0.748879


# Modelling

## Select Modelling Techniques

Recordando nuestro objeto de estudio, buscamos a través de grupos poblacionales agrupados por sitio donde más frecuentan sus compras, presentar posibles promociones que benefician a nuestros clientes, a través del medio de pago que emplean en dichos sitios. 

Ahora bien, dado que necesitamos sugerir las promociones para los individuos, estas estarán representadas por sitios, recordando que la base de datos fue ajustada de esa manera anteriormente. Teniendo en cuenta que las promociones se harán de acuerdo con las variables que tenemos como tipos de tarjetas, franjas horarias o dias de la semana, la metodología que vamos a emplear será K-Means, en donde el algoritmo nos permitirá identificar los puntos que más tengan similitud. 

Sabiendo que esta metodología necesita un K optimo donde minimiza la varianza entre puntos y la maximiza entre centroides, agrupandolos de acuerdo a dicho K, partimos de los datos originales para hallar este K, primero tomamos números del 1 al 15, con los que trazaremos los valores de inercia y poder hallar el k óptimo para los clusters.


## Testing Design

....

## Build Model

El modelo que vamos a utilizar será K-Means, ahora bien, como lo mencionamos anteriormente, este algoritmo tiene dos parámetros importantes, el primero el K que vamos a emplear, el cual será seleccionado bajo el método de codo, el cual utiliza la distancia media de las observaciones a su centroide. En otras palabras, busca fijar las distancias intra-cluster, en donde a medida que el K es mayor la varianza tiende a disminuir, lo que significaría que los clústeres son más compactos.

### Método de codo

In [11]:
k = list(range(1,15))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [12]:
# Crear una lista vacía para almacenar los valores de inertia
inertia = []

In [13]:
# Bucle for para calcular la inercia de cada K
for i in k:
    model = KMeans(n_clusters = i, random_state = 0)
    model.fit(df_infobanca_scaled)
    inertia.append(model.inertia_)

inertia

[1728.0,
 695.8366186458875,
 431.4127361062937,
 297.44037166632853,
 198.1877527289729,
 127.10795536254784,
 90.23584995838549,
 58.09242712561696,
 42.11419486615875,
 33.81132904640281,
 27.520495640642217,
 21.820193713615904,
 17.528704505216194,
 13.79981390367902]

In [14]:
# Crear un diccionario con los datos para trazar la curva del Codo
elbow_data_original = {
    'k': k,
    'inertia': inertia
}

# Crea un DataFrame con los datos para trazar la curva del Codo
df_elbow_data_original = pd.DataFrame(elbow_data_original)
df_elbow_data_original

,k,inertia
0,1,1728.000000
1,2,695.836619
2,3,431.412736
3,4,297.440372
4,5,198.187753
5,6,127.107955
6,7,90.235850
7,8,58.092427
8,9,42.114195
9,10,33.811329


In [15]:
# Observamos los valores de inercia en un gráfico de línea para encontrar el K óptimo

elbow_plot_original = df_elbow_data_original.hvplot.line(
    title = 'Elbow Plot: Original Data',
    x = 'k',
    xlabel = 'k',
    xticks = k,
    y = 'inertia',
    ylabel = 'Inertia'
)

elbow_plot_original

:Curve   [k]   (inertia)

A partir del gráfico, el mejor valor de K parece ser el K=6, el cual será el parametro a utilizar como número de clusteres que se formaran.

### Clusters ~ K-Means

Ahora armaremos clusters de los sitios de consumo más frecuentes usando el K=6 que encontramos en el paso anterior.

In [16]:
# Inicializar el modelo K-Means usando el mejor valor para k
model = KMeans(n_clusters=6)
model.fit(df_infobanca_scaled) # Ajuste del modelo

KMeans(n_clusters=6)

In [17]:
# Prediga los grupos para agrupar los sitios de consumo usando los datos escalados
clusters_original = model.predict(df_infobanca_scaled)

# Ver la matriz resultante de valores de clúster.
clusters_original

array([0, 2, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
      dtype=int32)

In [18]:
# Crear una copia del DataFrame
df_clusters_original = infobanca.copy()

Una vez tenemos los Clusteres que vamos a asignara a las observaciones (sitios), vamos a pegar ese vector columna con dicha información con nuestra base de datos original.

In [19]:
# Agregue una nueva columna al DataFrame con los grupos predichos
df_clusters_original['Cluster'] = clusters_original

# Mostrar datos de muestra
df_clusters_original['Cluster']

Sitio_consumo_masfrecuente
ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA    0
AEROLINEAS                                                                           2
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine                        0
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD                                      0
AGENCIAS DE VIAJE y OPERADORES DE TURISMO                                            3
                                                                                    ..
VENTA DE  ARTICULOS MEDICOS, ODONTOLOGICOS, ORTOPEDICOS                              0
VENTA DE  COMIDA RAPIDA                                                              0
VENTA DE BOTES Y ACCESORIOS PARA ACTIVIDADES ACUATICAS                               0
VENTA DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS                                        0
VIVEROS Y ALMACENES PARA JARDINERIA                                                  0
Name: Cluster, L

In [20]:
cluster_plot_original = df_clusters_original.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_manana',
    xlabel = 'porcentaje_manana',
    y = 'porcSABADO',
    ylabel = 'porcSABADO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original

:NdOverlay   [Cluster]
   :Scatter   [porcentaje_manana]   (porcSABADO,Sitio_consumo_masfrecuente)

### Optimizar clústeres con análisis de componentes principales

Otra metodología que se puede utilizar con el fin de contrastar los resultados del modelamiento anterior es la optimización de los clústeres, pero bajo un análisis de componentes principales, antes de correr el modelo. Este método lo que nos permitirá será reducir las variables que tengamos a un número definido a discreción del investigador, en este caso dado que tenemos 3 categorías en las que podemos agrupar toda la información (Tipo de tarjeta por franquicias, franja horaria, Noche o Día y dias de la semana) el número de componentes principales será 3.

In [21]:
pca_model = PCA(n_components=3) #Creación del algoritmo
pca_data = pca_model.fit_transform(df_infobanca_scaled) # Reducción a tres CP
pca_data[:5]

array([[-1.47392774, -0.05125033, -0.04254504],
       [11.01384135,  2.82308676, -0.63776749],
       [-1.29306492, -0.0518408 , -0.04859666],
       [-1.47486316, -0.05086348, -0.04259199],
       [ 3.51468205, -0.34977738,  0.19919799]])

In [22]:
# Recuperar la varianza explicada para determinar cuánta información
# se puede atribuir a cada componente principal.
display(pca_model.explained_variance_ratio_)
display(pca_model.explained_variance_)
display(pca_model.explained_variance_ratio_.sum()) # Varianza explica por los tres componentes principales 

array([0.82942008, 0.09102839, 0.05465898])

array([13.39474668,  1.47006603,  0.88271705])

0.975107456710012

Dado que ya tenemos los datos reducidos a tres componentes principales vamos a crear un DataFrame que los contenga. 

In [23]:
df_pca_data = pd.DataFrame(pca_data)
df_pca_data["Sitio_consumo_masfrecuente"] = infobanca.index
df_pca_data = df_pca_data.set_index('Sitio_consumo_masfrecuente')

# Mostrar datos de muestra
df_pca_data.head(5)

,0,1,2
Sitio_consumo_masfrecuente,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",-1.473928,-0.051250,-0.042545
AEROLINEAS,11.013841,2.823087,-0.637767
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,-1.293065,-0.051841,-0.048597
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,-1.474863,-0.050863,-0.042592
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,3.514682,-0.349777,0.199198


Una vez tenemos los datos bajo la metodologia ACP (Analisis de componentes principales), vamos a utilizar estos datos para calcular el K óptimo que se requiere a la ahora de correr nuestro modelo K-Means

In [24]:
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [25]:
inertia2 = []
for i in k:
    kmeans_model_pca = KMeans(n_clusters=i, random_state=1)
    kmeans_model_pca.fit(df_pca_data)
    inertia2.append(kmeans_model_pca.inertia_)

inertia2

[1684.9856851949023,
 653.3510971856781,
 390.95696027859645,
 260.96411543099885,
 162.26356248533057,
 94.25851690252813,
 59.47913065836612,
 30.982805283611196,
 20.774389722802294,
 14.576967329405088,
 9.899365249019564,
 7.3917857057815795,
 5.666809365920917,
 4.4666372927332105]

In [26]:
elbow_data_pca = {
    'k': k,
    'inertia':inertia2
}
df_elbow_data_pca = pd.DataFrame(elbow_data_pca)
df_elbow_data_pca

,k,inertia
0,1,1684.985685
1,2,653.351097
2,3,390.956960
3,4,260.964115
4,5,162.263562
5,6,94.258517
6,7,59.479131
7,8,30.982805
8,9,20.774390
9,10,14.576967


In [27]:
elbow_plot_pca = df_elbow_data_pca.hvplot.line(
    title='Elbow Plot: PCA',
    x = 'k',
    xticks = k,
    xlabel = 'k',
    y = 'inertia',
    ylabel = 'inertia'
)

elbow_plot_pca

:Curve   [k]   (inertia)

Teniendo en cuenta la gráfica de codo utilizando los datos calculados por componentes principales, notamos que tiene una similitud a la gráfica con los datos originales, por lo que seguiremos utilizando el mismo K, que en este caso será K = 6. 

In [28]:
kmeans_model_pca = KMeans(n_clusters=6)
kmeans_model_pca.fit(df_pca_data)

KMeans(n_clusters=4)

In [29]:
clusters_pca = kmeans_model_pca.predict(df_pca_data)
clusters_pca

array([0, 1, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
      dtype=int32)

In [30]:
df_clusters_pca = infobanca.copy() #DataFrame PCA
df_clusters_pca['Cluster'] = clusters_pca
df_clusters_pca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Cluster
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",1.03,0.00,0.00,0.00,0.00,0.00,1.03,0.00,0.00,0.00,0.00,0.58,0.00,0.00,0.00,0.45,0
AEROLINEAS,2186.26,711.13,1435.61,453.30,31.65,97.49,1491.06,2369.27,1055.11,447.30,659.29,780.10,803.88,779.99,853.69,591.19,1
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,47.32,5.80,20.99,2.36,0.16,0.44,21.66,37.81,17.59,11.69,5.22,11.81,13.45,11.16,7.65,16.08,0
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,0.62,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.62,0.00,0
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,1400.71,111.41,525.88,58.49,8.17,32.83,680.49,1265.26,191.74,124.75,329.84,363.96,331.60,343.42,363.76,280.15,3


In [31]:
cluster_plot_original2 = df_clusters_pca.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_mastercard_nacional',
    xlabel = 'porcentaje_mastercard_nacional',
    y = 'porcDOMINGO',
    ylabel = 'porcDOMINGO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original2

:NdOverlay   [Cluster]
   :Scatter   [porcentaje_mastercard_nacional]   (porcDOMINGO,Sitio_consumo_masfrecuente)

In [32]:
cluster_plot_original2 = df_clusters_pca.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_manana',
    xlabel = 'porcentaje_manana',
    y = 'porcSABADO',
    ylabel = 'porcSABADO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original2

:NdOverlay   [Cluster]
   :Scatter   [porcentaje_manana]   (porcSABADO,Sitio_consumo_masfrecuente)

# Evaluación 